In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
! pip install regex 

In [ ]:
import pandas as pd
data = pd.read_csv("gdrive/My Drive/AI-project/MyPersonality.csv")
#data = pd.read_csv("gdrive/My Drive/AI-project/reddit.tsv",sep='\t')
print(data.shape)

(1013000, 4)


In [ ]:
#clean dataset
import sys
import regex as re
import tokenize
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
import unicodedata 
from string import punctuation
import fileinput
from itertools import groupby, product
from datetime import datetime
debut=datetime.now()

tokens_twitter = {}
f = open("gdrive/My Drive/AI-project/glove.twitter.27B.25d.txt", 'r', encoding="utf8")
key=-1
for line in f:
    key+=1
    (val, vector) = line.split(" ", 1)
    tokens_twitter[int(key)] = val





FLAGS = re.MULTILINE | re.DOTALL
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
def hashtag(text): #hashtag
    text = text.group()
    hashtag_body = text[1:]
    if hashtag_body.isupper():
        result = "<hashtag> {} <allcaps>".format(hashtag_body)
    else:
        result = " ".join(["<hashtag>"] + re.split(r"(?=[A-Z])", hashtag_body, flags=FLAGS))
    return result

def allcaps(text): #mot tout en maj
    text = text.group()
    return text.lower() + " <allcaps>"


def load_dict_emojis():
    
    return {
        ":‑)":"<smile>",
        "=]]":"<smile>",
        ":]":"<smile>",
        ":}":"<smile>",
        ":o)":"<neutralface>",
        "=]":"<smile>",
        "=)":"<smile>",
        ":D":"<smile>",
        "xD":"<lolface>",
        "XD":"<lolface>",
        'xd':"<lolface>",
        ":[":"<sadface>",
        "=[[":"<sadface>",
        ":{":"<sadface>",
        ":@":"<sadface>",
        ":'-(":"<sadface>",
        "^^":"<smile>",
        "¬_¬":"<neutralface>",
        "o_O":"<neutralface>",
        "o_o":"<neutralface>",
        "¤_¤":"<smile>",
        "^o^":"<smile>",
        "^_^":"<smile>"
        
        }
    
    
def emoji_to_text(text):
    emojis = load_dict_emojis()  
    words = text.split()
    reformed = [emojis[word] if word in emojis else word for word in words]
    return " ".join(reformed)



def remove_punctuation(text):
    punctuation=['"',"'", "’" ]
    return ''.join(c for c in text if c not in punctuation)



def remove_consecutive_dups(s):
    return re.sub(r'(?i)(.)\1+', r'\1', s)

def all_consecutive_duplicates_edits(word, max_repeat=float('inf')):
    chars = [[c*i for i in range(min(len(list(dups)), max_repeat), 0, -1)]
             for c, dups in groupby(word)]   
    elong=False   
    for c in chars:
      if len(c)>2:
        elong=True

    return (map(''.join, product(*chars)),elong)


        
        

def clean(text):
    # Different regex parts for smiley faces
    eyes = r"[8:=;]"
    nose = r"['`\-o]?"
    eyeleft = r"[>*^o\-¤¬]" 
    mouth = r"[_o\.]?"

    # function so code less repetitive
    def re_sub(pattern, repl):
        return re.sub(pattern, repl, text, flags=FLAGS)
    
    
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>") #url
    text = text.replace("<PERSON>", "<user>")
    text = text.replace("<URL>", "<url>")
    
    text = re_sub(r"([A-Z]){2,}", allcaps) #mettre tout en minuscule
    text=text.lower()    
    text = re_sub(r"\.[ ]+\.", "..") 
    text = text.replace("‘", "'")

    
    text = re.sub(r"-{2,}", r"-", text) 
    text = re.sub(r"_{2,}", r"_", text)

    text = re.sub(r"([.]){4,}", r"… <repeat>", text)
    text = re.sub(r"([.]){2,3}", r"…", text)
    
    
    text = text.replace("“", '"')
    text = text.replace("”", '"')
    text = text.replace("`", "'")
    text = text.replace("¸¸", ",")
    text = text.replace("pl0x", "plox")
    text = re_sub(r"https?:\/\/\S+\b|www\.(\w+\.)+\S*", "<url>")
    
    text = text.replace("*propname*", "<user>")
    
    
    text = text.replace("*propname <allcaps>*", "<user>")
    text = re_sub(r"@\w+", "<user>")
    emojis = load_dict_emojis()  
    text = emoji_to_text(text) 
    text = re_sub(r"{}{}[)dD3\]]+|[\[(dD]+{}{}".format(eyes, nose, nose, eyes), "<smile>")

    text = re_sub(r"[>*\^\-¬][_o]{1}[<*\^\-¬]", "<smile>")
    text = re_sub(r"[o¤0][_]{1}[o0¤]", "<smile>")
    text = re_sub(r"\^\^", "<smile>#")
    eyeleft = r"[>*\^o\-¤¬]" 
    mouth = r"[_o\.]?"


    text = re_sub(r"{}{}p+".format(eyes, nose), "<lolface>")
    text = re_sub(r"x[d]+|[d]+x", "<lolface>")
    text = re_sub(r":[o]+|[o]+:", "<neutralface>")
    text = re_sub(r":[}]+|[{]+:", "<smile>")
    text = re_sub(r":[{]+|[}]+:", "<sadface>")
    text = re_sub(r"{}{}\(+|\)+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}@+|@+{}{}".format(eyes, nose, nose, eyes), "<sadface>")
    text = re_sub(r"{}{}[+|]+{}{}".format(eyes, nose, nose, eyes), "<sadface>")

    text = re_sub(r"/"," / ")
    
    text = re_sub(r"{}{}[\/|l*]".format(eyes, nose), "<neutralface>")
    text = re_sub(r"<3","<heart>")
    text = re_sub(r"[-+]?[.\d]*[\d]+[:,.\d]*", "<number>")
    text = re_sub(r"#\S+", hashtag)
    text = re_sub(r'([a-z])_', r'\1 ')  
    text = re_sub(r'_([a-z])', r' \1')  
    text = re_sub(r'([^a-z])\1+', r'\1')      
    text = re_sub(r"a[_*\.\-]s", "ass")
    text = re_sub("( |^)f[_*\.\-]( |$)", " fuck ")
    text = re_sub("( |^)[_*\.\-]up( |$|[^a-z])", " fuck up ")

    text = text.replace("t-shirt", "tshirt")      
                  
#    words = enchant.Dict("en")
#    is_known_word = words.check # a changer avec notre dictionnary
    
    from nltk.tokenize import TweetTokenizer
    tokenizer = TweetTokenizer()
    regex = re.compile(r"(.)\1{2}")

    if regex.search(text):
      text_shorten=[]
      for t in tokenizer.tokenize(text):
        if (t not in tokens_twitter.values()) :
          if all_consecutive_duplicates_edits(t)[1]:
            text_shorten.append(remove_consecutive_dups(t)+' <elong> ')
          else:
            text_shorten.append(t)
        else:    
          text_shorten.append(t)
      text=' '.join(text_shorten)



    text = re.sub(r'(\w+ )(\1)+', r'\1<repeat> ', text) #les mots qui se repete ex: bye bye
    text = re.sub(r'\W(\w+)(\1)+\W', r' \1 <repeat> ', text)
    text = re.sub(r'^(\w+)(\1)+', r' \1 <repeat> ', text)
    text = re.sub(r'(\w+ )(\1)+', r'\1<repeat> ', text) #les mots qui se repete ex: bye bye

    string=''
    for token in text.split(' '):
      m=re.search('(.*?)(([a-z]\.){2,})(.*)', token)
      if m:
        replacement=''.join(m.group(2).split('.'))
        token=m.group(1)+replacement+m.group(4)
        
      string+=token+' '  
    text=string

    string=''
    for token in text.split(' '):
      m=re.search('(.*?)(([a-z]-){2,})(.*)', token)
      if m:
        replacement=''.join(m.group(2).split('-'))
        token=m.group(1)+replacement+m.group(4)
      string+=token+' ' 
      text=string

    
    text = re.sub(r'([A-Za-z])-([A-Za-z])', lambda m: '%s %s' % (m.groups()), text)#ex: half-done=>half done
    text = re.sub(r'([A-Za-z])_([A-Za-z])', lambda m: '%s %s' % (m.groups()), text)#ex: half_done=>half done


    text = re.sub(r'([A-Za-z]);([A-Za-z]*)', r'\1 ;\2', text)#ex:  do;e => do ;e
    text = re.sub(r'([A-Za-z]):([A-Za-z]*)', r'\1 :\2', text)#ex:  do:e => do :e


    text = text.replace("'", "")
    text = text.replace("’", "")

    text = re.sub(r'([A-Za-z])\.([A-Za-z])', r'\1. \2', text)#ex: elle.lui=> elle. lui

    text = re.sub(r"\b(\S*?)(.)\2{2,}(\S*?)", r"\1\2",text)

    text = text.replace("<-", " ")
    text = text.replace("->", " ")
    text = text.replace(r" > < "," >< ")
    text = text.replace(r" :\ "," <sadface> ")
    text = text.replace(r" t t "," <sadface> ")
    text = text.replace(r" n n "," <smile> ")
    text = text.replace(r" n <repeat> "," <smile> ")
    text = text.replace(r" m <repeat> "," mmm ")
    text = text.replace(r" w <number> t "," woot ")
    text = text.replace(r" l <repeat>"," ll")
    text = text.replace(r" <allcaps> s "," s <allcaps> ")
    text = text.replace(r" <allcaps> t "," t <allcaps> ")
    text = text.replace("<lb>", " ")  
    text = re_sub(r"\s+", " ")
    return text



#Clean dataset 
pd.set_option('display.max_columns', None)
i=-1
for s in data['text']: #selftext
    i+=1
    print(i)
    s= clean(s)
    data.at[i, 'text']=s



#Emoji to text 
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()
import demoji
demoji.download_codes()
for s in data["selftext"]:
    i+=1
    tokens= tokenizer.tokenize(s)
    chaine=''
    for t in tokens:
      corr=t
      if demoji.findall(t)!={}:
        exclude = set(string.punctuation)
        emojis=demoji.findall(t)
        if(t in emojis):
          corr = ''.join(ch for ch in emojis[t].lower() if ch not in exclude)
          

      chaine=chaine+corr+' '
           
    data.at[i, 'text']=chaine



data.to_csv(r'gdrive/My Drive/AI-project/mypersonality_final.csv', index = False) 
#data.to_csv(r'gdrive/My Drive/AI-project/unlab_reddit.csv', index = False)  
